## Data Souces:

* Foursquare for food store types and locations.

* Chicago zip codes
    * https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Chicago%20(IL)1.html

* Us population by zipcode
    * https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv

* lat long of us zipcodes
    * https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/
    * https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

* Folium for mapping and plotting

* Foursquare in particular categories
    * https://developer.foursquare.com/docs/build-with-foursquare/categories/
    * From the Resaurant group
        * Fast Food Restaurant
    * From the Food and Dining Shop group
        * Liquor Store
        * Farmers Market
        * Organic Grocery
        * Supermarket
        * Grocery Store

## Data Usage:
The data will be combinded to generate a data set consiting of a count of the Facebook groups with Grocery stores and like as good things while Fast Food and Liquor Stores counting as bad each as a ratio of population then it is time for stats to see what can be learned and what models can be built


**Setup Testing and intial exploration of data**

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import geocoder
import folium 


Collect and assemble the zip code data frames

In [5]:
URL = 'https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B'
zipCodesDF = pd.read_csv(URL, sep=';')
zipCodesDF.shape

(43191, 8)

In [33]:
zipCodesDF.head(10)


,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,48834,Fenwick,MI,43.141649,-85.049480,-5,1,"43.141649,-85.04948"
1,55304,Andover,MN,45.254715,-93.286520,-6,1,"45.254715,-93.28652"
2,55422,Minneapolis,MN,45.014764,-93.339650,-6,1,"45.014764,-93.33965"
3,29079,Lydia,SC,34.296064,-80.113190,-5,1,"34.296064,-80.11319"
4,29390,Duncan,SC,34.888237,-81.969020,-5,1,"34.888237,-81.96902"
5,29446,Green Pond,SC,32.683083,-80.560740,-5,1,"32.683083,-80.56074"
6,30018,Jersey,GA,33.717989,-83.801451,-5,1,"33.717989,-83.801451"
7,30093,Norcross,GA,33.909952,-84.179400,-5,1,"33.909952,-84.1794"
8,30417,Claxton,GA,32.162077,-81.909840,-5,1,"32.162077,-81.90984"
9,30627,Carlton,GA,33.994139,-82.978190,-5,1,"33.994139,-82.97819"


In [19]:
URL = 'https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv'
zipcodePopulation = pd.read_csv(URL)

zipcodePopulation.head(10)

,Zip Code ZCTA,2010 Census Population
0,1001,16769
1,1002,29049
2,1003,10372
3,1005,5079
4,1007,14649
5,1008,1263
6,1009,741
7,1010,3609
8,1011,1370
9,1012,661


In [30]:
URL = 'https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Chicago%20(IL)1.html'
rawPage = requests.get(URL, 'html.parser')
soup = BeautifulSoup(rawPage.text, 'html.parser')
table = soup.find_all('table')[0]
chicagoZip = pd.read_html(str(table))[0]

chicagoZip.head(10)
foo = chicagoZip[0].str.split(' ', 1, expand=True)
foo.columns=["ZipCode", "Neighborhood"]
#foo.dtypes
foo["ZipCode"] = foo["ZipCode"].astype(int)
#foo.head(10)
completeDF = foo.merge(zipCodesDF, left_on='ZipCode', right_on='Zip', how='left')

completeDF = completeDF.merge(zipcodePopulation, left_on='ZipCode', right_on='Zip Code ZCTA', how='left')
completeDF = completeDF.drop(['Zip', 'geopoint', 'Daylight savings time flag', 'Timezone','Zip Code ZCTA'], axis=1).dropna()
completeDF.head(30)

,ZipCode,Neighborhood,City,State,Latitude,Longitude,2010 Census Population
1,60002,Antioch,Antioch,IL,42.466170,-88.09995,24299.0
2,60002,Old Mill Creek,Antioch,IL,42.466170,-88.09995,24299.0
3,60002,Old Mill Crk,Antioch,IL,42.466170,-88.09995,24299.0
4,60002,Wadsworth,Antioch,IL,42.466170,-88.09995,24299.0
5,60004,Arlington Heights,Arlington Heights,IL,42.108428,-87.97723,50582.0
6,60005,Arlington Heights,Arlington Heights,IL,42.069327,-87.98464,29308.0
8,60007,Elk Grove Village,Elk Grove Village,IL,42.005978,-87.99847,33820.0
9,60008,Rolling Meadows,Rolling Meadows,IL,42.075060,-88.02508,22717.0
11,60010,Barrington,Barrington,IL,42.160791,-88.15231,44095.0
12,60010,Barrington Hills,Barrington,IL,42.160791,-88.15231,44095.0


In [32]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
search_query = 'Food'
radius = 2000
postCodeList = []
jsonResult = []

lat = '42.265643'
long = '-88.31664'
searchURL = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
result = requests.get(searchURL).json()
result


#df_with_geo.iloc[0]
#lat = df_with_geo.iloc[0]['Latitude']
#long = df_with_geo.iloc[0]['Longitude']
#postcode =  df_with_geo.iloc[0]['Postal Code']
#for row in df_with_geo.itertuples(index=True, name='Pandas'):
#    lat = row.Latitude
#    long = row.Longitude
#    postCode = row.PostalCode
#    postCodeList.append(postCode)
#    searchURL = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
#    result = requests.get(searchURL).json()
#    jsonResult.append(result)

#toAdd = list(zip(postCodeList, jsonResult))
#df_temp = pd.DataFrame(toAdd, columns = ['PostalCode', 'jsonResult'])
#df_Farmers = df_with_geo.merge(df_temp, left_on='PostalCode', right_on='PostalCode')
#toAdd

{'meta': {'code': 200, 'requestId': '5f352f6829b3b454743c8789'},
 'response': {'venues': [{'id': '4daa18116a2303012f06507a',
    'name': 'Crystal Lake Food and Liquor',
    'location': {'address': '540 E Terra Cotta Ave',
     'lat': 42.249189,
     'lng': -88.300545,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.249189,
       'lng': -88.300545},
      {'label': 'entrance', 'lat': 42.249106, 'lng': -88.300658}],
     'distance': 2261,
     'postalCode': '60014',
     'cc': 'US',
     'city': 'Crystal Lake',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['540 E Terra Cotta Ave',
      'Crystal Lake, IL 60014',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d186941735',
      'name': 'Liquor Store',
      'pluralName': 'Liquor Stores',
      'shortName': 'Liquor Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_liquor_',
       'suffix': '.png'},
      'primary': True}],
    'referralId'